In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-03 18:35:45--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-12-03 18:35:45 (5.99 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark_Music").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [5]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

4751577
4750852
4750852


In [6]:
from pyspark.sql.functions import to_date

review_id_table = df.select('review_id', 'customer_id', 'product_id', 'product_parent', to_date('review_date', "yyyy-MM-dd").alias('review_date'))
review_id_table.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1005BINTGKCVE|   18789304|B000002J6H|     480387571| 2013-01-29|
|R10067SN6FSII1|   32818060|B00005BCY9|     926227231| 2003-01-19|
|R100NGDZ6CAL8E|   32318800|B0000AB13N|     711057433| 2003-08-04|
|R100OLMAO039IL|   50673975|B000002SZA|     632006013| 2009-09-10|
| R100OMDUXE5VZ|   52503259|B000056KSU|     155726790| 2003-02-08|
|R100RJOC7TCXN6|   47602845|B005G492CC|     819279192| 2014-06-03|
|R10182KW3O6486|   52776942|B000031WEY|     719850793| 2004-04-23|
|R101B8W76YQILD|   12264663|B000BM7YYW|     364079538| 2006-03-10|
|R101K97ELZLSDK|   51870288|B00006NSDJ|     319548298| 2003-07-28|
|R101MMM4IXONBO|   43769541|B00C4XYEFS|     319009128| 2014-03-27|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [7]:
products = df.select('product_id', 'product_title')
products = products.dropDuplicates()
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B003JYOFIW|Something for Eve...|
|B00049QMG8|  Mississauga Goddam|
|B0000014Y7|Sherlock Holmes: ...|
|B000EXOAAO|                  IV|
|B0000033QY|Live at Carnegie ...|
|B0001HN5HM|That's What I'm T...|
|B000002BOM|Nick at Nite's Cl...|
|B000PHX0ZA|        Oboe Sonatas|
|B0035WTE2U| Iron Man 2 (CD/DVD)|
|B00004YNES|    The Duo Sessions|
|B000092Q7L|Take This To Your...|
|B000026KGC|The Book Of Talie...|
|B00007BH98|Earl Goes to the ...|
|B000005ZXL|The Ultimate Hist...|
|B000GCG60K|Shaken By a Low S...|
|B00005UK0N|Asleep in the Bac...|
|B000VI04HK|         R.E.M. Live|
|B000002L8C|                 Tim|
|B000008FCR|Electric Sugarcub...|
|B00004VW14|The End of the Ra...|
+----------+--------------------+
only showing top 20 rows



In [8]:
customers = df.select('customer_id')
customers = customers.groupBy('customer_id').count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   51893555|             1|
|   12945148|             5|
|   38897684|            21|
|   15261709|             3|
|   53036614|           125|
|   50509124|             2|
|   47996333|            23|
|   13560803|             3|
|   51680053|            23|
|   52037325|            14|
|   30682568|            58|
|   35871172|            88|
|   52599576|             3|
|   39162192|             6|
|   46859391|             1|
|   22421107|             6|
|   14219983|             4|
|   10944849|             1|
|   45936211|             1|
|    1592147|            23|
+-----------+--------------+
only showing top 20 rows



In [9]:
vine_table = df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine')
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1005BINTGKCVE|          5|            0|          0|   N|
|R10067SN6FSII1|          4|            0|          0|   N|
|R100NGDZ6CAL8E|          1|            2|          2|   N|
|R100OLMAO039IL|          3|            2|          2|   N|
| R100OMDUXE5VZ|          4|            0|          0|   N|
|R100RJOC7TCXN6|          5|            2|          2|   N|
|R10182KW3O6486|          4|            5|          6|   N|
|R101B8W76YQILD|          4|            5|          5|   N|
|R101K97ELZLSDK|          4|            7|          9|   N|
|R101MMM4IXONBO|          5|            0|          0|   N|
|R101SUFE1N98DO|          5|            3|          6|   N|
|R101Y7O5QCTMH4|          4|            5|          7|   N|
|R1020J9O3H1F52|          5|            0|          0|   N|
|R1024GZRHXWW2F|          1|            

#**RDS**

In [10]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<database>:5432/MusicDB"
config = {"user":"<user>",
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

In [11]:
# Write DataFrame to table

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [12]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [13]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [14]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)